In [1]:
#!wget https://raw.githubusercontent.com/romeokienzler/developerWorks/master/model.h5
#!wget https://raw.githubusercontent.com/romeokienzler/developerWorks/master/model.json

In [2]:
!rm watsoniotp.*
!wget https://raw.githubusercontent.com/romeokienzler/developerWorks/master/watsoniotp.healthy.phase_aligned.pickle
!wget https://raw.githubusercontent.com/romeokienzler/developerWorks/master/watsoniotp.broken.phase_aligned.pickle

--2017-12-12 15:14:03--  https://raw.githubusercontent.com/romeokienzler/developerWorks/master/watsoniotp.healthy.phase_aligned.pickle
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.48.133
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.48.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 194639 (190K) [text/plain]
Saving to: ‘watsoniotp.healthy.phase_aligned.pickle’

100%[======================================>] 194,639     --.-K/s   in 0.009s  

2017-12-12 15:14:03 (21.3 MB/s) - ‘watsoniotp.healthy.phase_aligned.pickle’ saved [194639/194639]

--2017-12-12 15:14:03--  https://raw.githubusercontent.com/romeokienzler/developerWorks/master/watsoniotp.broken.phase_aligned.pickle
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.48.133
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.48.133|:443... connected.
HTTP request sent, awaiting response... 200 OK


In [3]:
import numpy as np
from numpy import concatenate
from matplotlib import pyplot
from pandas import read_csv
from pandas import DataFrame
from pandas import concat
import sklearn
from  sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.callbacks import Callback
from keras.models import Sequential
from keras.layers import LSTM, Dense, Activation
from keras.models import model_from_json
import pickle
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import ibmiotf.application
import time
from Queue import Queue
import numpy as np
import sys
%matplotlib inline

Using TensorFlow backend.


In [4]:
def scaleData(data):
    # normalize features
    scaler = MinMaxScaler(feature_range=(0, 1))
    return scaler.fit_transform(data)

In [5]:
timesteps = 10
dim = 3
samples = 3000

In [6]:
losses = []

def handleLoss(loss):
        global losses
        losses+=[loss]
        print loss

In [7]:
class LossHistory(Callback):
    def on_train_begin(self, logs={}):
        self.losses = []

    def on_batch_end(self, batch, logs={}):
        self.losses.append(logs.get('loss'))
        handleLoss(logs.get('loss'))

In [8]:
# design network
model = None

def reloadModel():
    global model
    json_file = open('model.json', 'r')
    loaded_model_json = json_file.read()
    json_file.close()
    model = model_from_json(loaded_model_json)
    # load weights into new model
    model.load_weights("model.h5")
    print("Loaded model from disk")
    model.compile(loss='mae', optimizer='adam')

reloadModel()

def train(data):
    data.shape = (300, 10, 3)
    model.fit(data, data, epochs=5, batch_size=72, validation_data=(data, data), verbose=0, shuffle=False,callbacks=[LossHistory()])
    data.shape = (3000, 3)

def score(data):
    data.shape = (300, 10, 3)
    yhat =  model.predict(data)
    yhat.shape = (3000, 3)
    return yhat

Loaded model from disk


In [9]:
# USA options = {"org": "oqyrc5", "id": "anything", "auth-method": "apikey", "auth-key": "a-oqyrc5-gqjx6gmrpc", "auth-token": "XZmvYr0i7-@JoJotiB"}
# GER options = {"org": "vpypwh", "id": "systemml", "auth-method": "apikey", "auth-key": "a-vpypwh-uo4xdahvcy", "auth-token": "BwEwDnKSzHEgr?IcEc"}
options = {"org": "vpypwh", "id": "systemml", "auth-method": "apikey", "auth-key": "a-vpypwh-uo4xdahvcy", "auth-token": "BwEwDnKSzHEgr?IcEc"}
client = ibmiotf.application.Client(options)
client.connect()

2017-12-12 15:14:22,642   ibmiotf.application.Client  INFO    Connected successfully: a:vpypwh:systemml


In [10]:
q = Queue(7000)
simulator_status = 'disablesimulator'

In [11]:
def myEventCallback(event):
    global simulator_status
    if event.event == 'switch':
        simulator_status = event.data["message"]
        client.publishEvent("0.16.2", "lorenz", "switch_confirm", "json", simulator_status)
    if event.event == 'osc':
        sample = event.data
        point = [sample["x"], sample["y"],sample["z"]]
        q.put(point)

client.deviceEventCallback = myEventCallback
client.subscribeToDeviceEvents("0.16.2", "lorenz", "switch")
client.subscribeToDeviceEvents("0.16.2", "lorenz", "osc")
client.publishEvent("0.16.2", "lorenz", "alert", "json", {'message' : 'no alert'})

True

In [12]:
def doNN(data):
    data_scaled = scaleData(data)
    train(data_scaled)
    yhat = score(data_scaled)
    data_scaled.shape = (3000, 3)

In [ ]:
do_reset = False
reset_start = 0

def handleLoss(loss):
    global do_reset
    global reset_start
    sys.stdout.write('*')
    sys.stdout.flush()
    myData={'loss' : str(loss)}
    client.publishEvent("0.16.2", "lorenz", "status", "json", myData)
    if loss > 0.08:
        print "loss > 0.08"
        do_reset = True
        reset_start = int(time.time())
        client.publishEvent("0.16.2", "lorenz", "alert", "json", {'message' : 'alert'})
    if do_reset:
        if reset_start + 30 < int(time.time()):
            print "resetting"
            client.publishEvent("0.16.2", "lorenz", "alert", "json", {'message' : 'no alert'})
            client.publishEvent("0.16.2", "lorenz", "cmd", "json", {'message' : 'stop'})
            reloadModel()
            do_reset = False
            reset_start = 0
        

In [ ]:
while True:
    while not q.empty() and simulator_status=='disablesimulator':
        if simulator_status == 'disablesimulator':
            sys.stdout.write('.')
            sys.stdout.flush()
            point = q.get()
            try:
                data
            except NameError:
                data = np.array(point)
            else:
                data = np.append(data,point)
            if data.size>=9000:
                data = np.reshape(data,(3000,3))
                print "Sending window downstream to the neural network..."
                doNN(data)
                print "Training finished..."
                del data
    if simulator_status == 'simulatehealthy':
        data_healthy = pickle.load(open('watsoniotp.healthy.phase_aligned.pickle', 'rb'))
        data_healthy = data_healthy.reshape(3000,3)
        print "Sending window downstream to the neural network..."
        doNN(data_healthy)
        print "Training finished..."
    if simulator_status == 'simulatebroken':
        data_broken = pickle.load(open('watsoniotp.broken.phase_aligned.pickle', 'rb'))
        data_broken = data_broken.reshape(3000,3)
        print "Sending window downstream to the neural network..."
        doNN(data_broken)
        print "Training finished......"
            


.....................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................Sending window downstream to the neural network...
*************************Training finished...
Sending window downstream to the neural network...
*************************Training finished...
Sending window downstream to the neural network...
*************************Training finished...
Sending window downstream to the neural network...
*************************Training finished...
Sending window downstream to the neural network...
*************************Training finished...
Sending window